### GBDT算法例子

编号 | 年龄（岁） | 体重（Kg） | 身高（标签值） 
:-: | :-: | :-: | :-: 
1 | 5 | 20 | 1.1 
2 | 7| 30 | 1.3 
3 | 21| 70 | 1.7 
4 | 30| 60 | 1.8 

#### 1. 初始化弱学习器

$L = arg \ min_{\gamma}\sum_{i=1}^{N} L(y_i,\gamma)$

$\frac{\partial L(y_i,\gamma)}{\partial \gamma} = \sum_{i=1}^{N} \frac{\partial(\frac{1}{2} [y-\gamma]^2)}{\partial \gamma} = \sum_{i=1}^{N}\gamma - y$

* 得：$\gamma = y = (1.1+1.3+1.7+1.8)/4 = 1.475$

#### 2. 迭代轮数m=1

* 残差计算：$r_{i1} = -[\frac{\partial L(y_i,f{(x_i)})}{\partial f{(x_i)}}]_{f(x) = f_0{(x)}}$

编号 | 真实值 | f0(x) | 残差
:-: | :-: | :-: | :-: 
1 | 1.1 | 1.475 | -0.375 
2 | 1.3| 1.475 | -0.175 
3 | 1.7| 1.475 | 0.225 
4 | 1.8| 1.475 | 0.325 

* 此时训练数据变为

编号 | 年龄（岁） | 体重（Kg） | 残差（标签值） 
:-: | :-: | :-: | :-: 
1 | 5 | 20 | -0.375 
2 | 7| 30 | -0.175 
3 | 21| 70 | 0.225 
4 | 30| 60 | 0.325 

#### 3. 寻找回归的最佳分割点

* 需要遍历每个特征的每个取值，从年龄特征到体重特征，遍历所有可能的划分，分别计算方差，找到使方差最小的那个划分点为最好的划分点：

划分点 | 小于划分点的样本 | 大于等于划分点的样本 | 总方差
:-: | :-: | :-: | :-: 
年龄5 | - | 1,2,3,4 | 0.082
年龄7 | 1| 2,3,4 | 0.047 
年龄21 | 1,2| 3,4 | 0.0125 
年龄30 | 1,2,3| 4 | 0.062 
体重20 | -| 1,2,3,4 | 0.082 
体重30 | 1| 2,3,4 | 0.047 
体重60 | 1,2| 3,4 | 0.0125 
体重70 | 1,2,4| 3 | 0.0867

* 这一步需要遍历所有特征的所有的取值。以上划分点是的总方差最小为0.0125有两个划分点：年龄21和体重60，所以随机选一个作为划分点，这里我们选年龄21。

#### 4. 拟合残差

* 需要给这两个叶子节点分别赋一个参数，来拟合残差。

$\gamma_{j1} = arg \ min_{\gamma}\sum_{x_i\in R_{j1}} L(y_i,f_0(x_i) + \gamma)$

$\gamma_{11} = -0.275$

$\gamma_{12} = 0.275$

#### 5. 得到强学习器

* 得到强学习器为：

$f_{1}(x) = f_{0}(x) + \sum_{j=1}^{2}\gamma_{j1} I$

* 输入值年龄小于21岁，输出$1.475-0.275 = 1.2$
* 输入值年龄大于21岁，输出$1.475+0.275 = 1.75$

#### 6. 迭代m轮后

* 最终得到的强学习器为：

$f(x) = f_{0}(x) + \sum_{m=1}^{M}\sum_{j=1}^{N}\gamma_{j1} I$

### Xgboost的优化

* 损失函数上的优化
    <br>[引入正则化，把损失函数进行二阶泰勒展开](#损失函数二阶展开)</br>
* 寻找分割点算法优化
    <br>[列采样、行采样](#列采样、行采样)</br>
    <br>[预排序](#预排序)</br>
    <br>[近似算法](#近似算法)</br>
    <br>[加权分位数示意图](#加权分位数示意图)</br>
*  健壮性上的优化
    <br>[稀疏感知](#稀疏感知)</br>

##### 损失函数二阶展开

* The tree ensemble model in Eq. (2) includes functions as parameters and cannot be optimized using traditional optimization methods in Euclidean space. Instead, the modelis trained in an additive manner. Formally, let ${\hat{y}}^{(t)}$ be the prediction of the i-th instance at the t-th iteration, we will need to add ft to minimize the following objective.

* 式（2）中的树系综模型包含函数作为参数，在欧氏空间中不能用传统的优化方法进行优化。相反，模型是以加法的方式训练的。形式上，假设yˆ（t）是第t次迭代时第i个实例的预测，我们将需要添加ft以最小化以下目标。

![avatar](pic/3.png)

![avatar](pic/4.png)

* This means we greedily add the ft that most improves our model according to Eq. (2). Second-order approximation can be used to quickly optimize the objective in the general setting [12].

* 这意味着我们贪婪地根据公式(2)添加了最能改进我们模型的ft。二阶近似可以用于在一般设置中快速优化目标

![avatar](pic/5.png)

* 其中![avatar](pic/6.png)

* are first and second order gradient statistics on the loss function. We can remove the constant terms to obtain the following simplified objective at step t.

* 是关于损失函数的一阶和二阶梯度统计量。在步骤t中，我们可以去掉常数项以获得以下简化目标。

![avatar](pic/7.png)

![avatar](pic/8.png)

* 对应的最优解是：

![avatar](pic/10.png)

* 可以得到当前节点的损失：

![avatar](pic/11.png)

* Eq (6) can be used as a scoring function to measure the quality of a tree structure q. This score is like the impurity score for evaluating decision trees, except that it is derived for a wider range of objective functions. Fig. 2 illustrates how this score can be calculated.

* 等式（6）可用作衡量树结构q质量的评分函数。该评分类似于评价决策树的杂质评分，只是它是为更广泛的目标函数推导的。图2说明如何计算该分数。

* Normally it is impossible to enumerate all the possible tree structures q. A greedy algorithm that starts from a single leaf and iteratively adds branches to the tree is used instead. Assume that IL and IR are the instance sets of left and right nodes after the split. Lettting I = IL ∪ IR, then the loss reduction after the split is given by

* 通常不可能枚举所有可能的树结构q。取而代之的是一个贪婪的算法，它从一个叶子开始，迭代地向树中添加分支。假设IL和IR是拆分后左右节点的实例集。Letting I=IL∪IR，则分割后的损失减少量由下式给出

![avatar](pic/12.png)

* 算法流程

![avatar](pic/29.png)

<br>[返回](#Xgboost的优化)</br>

##### 列采样、行采样

* Besides the regularized objective mentioned in Sec. 2.1, two additional techniques are used to further prevent over-fitting. The first technique is shrinkage introduced by Friedman [11]. Shrinkage scales newly added weights by a factor η after each step of tree boosting. Similar to a learning rate in tochastic optimization, shrinkage reduces the influence of each individual tree and leaves space for future trees to improve the model. The second technique is column (feature) subsampling. This technique is used in RandomForest [4,13], It is implemented in a commercial software TreeNet 4 for gradient boosting, but is not implemented in existing opensource packages. According to user feedback, using column sub-sampling prevents over-fitting even more so than the traditional row sub-sampling (which is also supported). The usage of column sub-samples also speeds up computations of the parallel algorithm described later.

* 除了第2.1节中提到的正则化目标外，还使用了另外两种技术来进一步防止过拟合。第一种技术是由Friedman[11]引入的收缩。在每一步提高树的重量后，收缩将增加一个因子。与模型优化中的学习率相似，收缩减小了每棵树的影响，为未来树改进模型留下了空间。第二种技术是列(特征)子抽样。这种技术在RandomForest中使用[4,13]，它是在一个用于梯度增强的商业软件TreeNet 4中实现的，但在现有的开源包中没有实现。根据用户反馈，使用列子抽样比传统的行子抽样(也支持行子抽样)更能防止过拟合。列子样本的使用也加速了后面描述的并行算法的计算。

<br>[返回](#Xgboost的优化)</br>

##### 预排序

* One of the key problems in tree learning is to find the best split as indicated by Eq (7). In order to do so, a split finding algorithm enumerates over all the possible splits on all the features. We call this the exact greedy algorithm. Most existing single machine tree boosting implementations, such as scikit-learn [20], R’s gbm [21] as well as the single machine version of XGBoost support the exact greedy algorithm. The exact greedy algorithm is shown in Alg. 1. It is computationally demanding to enumerate all the possible splits for continuous features. In order to do so efficiently, the algorithm must first sort the data according to feature values and visit the data in sorted order to accumulate the gradient statistics for the structure score in Eq (7).

* 树学习中的一个关键问题是如何找到Eq（7）所表示的最佳分割。为了做到这一点，分割查找算法会枚举所有特征上所有可能的分割。我们称之为精确贪婪算法。大多数现有的单机器树提升实现，如scikit learn[20]、R的gbm[21]以及XGBoost的单机器版本都支持精确的贪婪算法。精确的贪心算法如Alg所示。一。计算上要求枚举连续特征的所有可能分割。为了有效地实现这一点，算法必须首先根据特征值对数据进行排序，然后访问排序后的数据，以累积Eq（7）中结构得分的梯度统计。

<br>[返回](#Xgboost的优化)</br>

##### 近似算法

* The exact greedy algorithm is very powerful since it enumerates over all possible splitting points greedily. However, it is impossible to efficiently do so when the data does not fit entirely into memory. Same problem also arises in the distributed setting. To support effective gradient tree boosting in these two settings, an approximate algorithm is needed.

* 精确贪婪算法非常强大，因为它贪婪地搜索所有可能的分裂点。然而，当数据不能完全装入内存时，就不可能有效地这样做。同样的问题也出现在分布式设置中。为了在这两种情况下支持有效的梯度树增强，需要一种近似算法。

* We summarize an approximate framework, which resembles the ideas proposed in past literatures [17, 2, 22], in Alg. 2. To summarize, the algorithm first proposes candidate splitting points according to percentiles of feature distribution (a specific criteria will be given in Sec. 3.3). The algorithm then maps the continuous features into buckets split by these candidate points, aggregates the statistics and finds the best solution among proposals based on the aggregated statistics.

* 我们总结了一个近似的框架，类似于在过去的文献[17,2,22]中提出的想法。综上所述，算法首先根据特征分布的百分位数提出候选分裂点(具体标准将在第3.3节给出)。然后，该算法将连续的特征映射到由候选点分割的桶中，对统计数据进行汇总，并根据汇总的统计数据在建议中找到最佳的解决方案。

* There are two variants of the algorithm, depending on when the proposal is given. The global variant proposes all the candidate splits during the initial phase of tree construction, and uses the same proposals for split finding at all levels. The local variant re-proposes after each split. The global method requires less proposal steps than the local method. However, usually more candidate points are needed for the global proposal because candidates are not refined after each split. The local proposal refines the candidates after splits, and can potentially be more appropriate for deeper trees. A comparison of different algorithms on a Higgs boson dataset is given by Fig. 3. We find that the local proposal indeed requires fewer candidates. The global proposal can be as accurate as the local one given enough candidates.

* 该算法有两种变体，取决于何时给出建议。全局变量在树构造的初始阶段提出了所有的候选分割，并在所有层次上使用相同的分割查找建议。局部变体在每次分割后重新提出。全局方法比局部方法需要更少的建议步骤。但是，通常全局方案需要更多的候选点，因为候选点在每次分割后都没有细化。局部建议在分割后细化候选树，可能更适合更深的树。图3给出了不同算法在希格斯玻色子数据集上的比较。我们发现，地方提案的确需要较少的候选人。如果有足够的候选方案，全局方案可以和局部方案一样准确。

<br>[返回](#Xgboost的优化)</br>

##### 加权分位数示意图

* To solve this problem, we introduced a novel distributed weighted quantile sketch algorithm that can handle weighted data with a provable theoretical guarantee. The general idea is to propose a data structure that supports merge and prune operations, with each operation proven to maintain a certain accuracy level. A detailed description of the algorithm as well as proofs are given in the appendix.

* 为了解决这一问题，我们提出了一种新的分布式加权分位数草图算法，该算法能够在可证明的理论保证下处理加权数据。总体思路是提出一种支持合并和剪枝操作的数据结构，并证明每个操作都能保持一定的精度水平。附录中给出了算法的详细描述和证明。

<br>[返回](#Xgboost的优化)</br>

##### 稀疏感知

* In many real-world problems, it is quite common for the input x to be sparse. There are multiple possible causes for sparsity: 1) presence of missing values in the data; 2) frequent zero entries in the statistics; and, 3) artifacts of feature engineering such as one-hot encoding. It is important to make the algorithm aware of the sparsity pattern in the data. In order to do so, we propose to add a default direction in each tree node, which is shown in Fig. 4. When a value is missing in the sparse matrix x, the instance is classified into the default direction. There are two choices of default direction in each branch. The optimal default directions are learnt from the data. The algorithm is shown in Alg. 3. The key improvement is to only visit the non-missing entries Ik. The presented algorithm treats the non-presence as a missing value and learns the best direction to handle missing values. The same algorithm can also be applied when the non-presence corresponds to a user specified value by limiting the enumeration only to consistent solutions.

* 在许多实际问题中，输入x稀疏是很常见的。稀疏性有多种可能的原因：1）数据中缺少值；2）统计中经常出现零项；3）特征工程的结果，比如一个热编码。该算法对数据中稀疏模式的识别具有重要意义。为此，我们建议在每个树节点中添加一个默认方向，如图4所示。当稀疏矩阵x中缺少值时，实例将被分类为默认方向。每个分支中有两个默认方向的选择。从数据中学习最佳默认方向。算法如Alg所示。三。关键的改进是只访问未丢失的条目Ik。该算法将不存在视为缺失值，并学习处理缺失值的最佳方向。当不存在对应于用户指定的值时，也可以应用相同的算法，方法是将枚举限制为一致的解。

![avatar](pic/30.png)

<br>[返回](#Xgboost的优化)</br>

### lightGBM的优化
* 优化方法
    <br>[减少数据上的优化](#减少数据上的优化)</br>
    <br>[减少特征上的优化](#减少特征上的优化)</br>

##### 减少数据上的优化

* In AdaBoost, the sample weight serves as a good indicator for the importance of data instances. However, in GBDT, there are no native sample weights, and thus the sampling methods proposed for AdaBoost cannot be directly applied. Fortunately, we notice that the gradient for each data instance in GBDT provides us with useful information for data sampling. That is, if an instance is associated with a small gradient, the training error for this instance is small and it is already well-trained. A straightforward idea is to discard those data instances with small gradients. However, the data distribution will be changed by doing so, which will hurt the accuracy of the learned model. To avoid this problem, we propose a new method called Gradient-based One-Side Sampling (GOSS).

* 在AdaBoost中，样例权重可以很好地指示数据样本的重要性。但是在GBDT中，没有天然的样本权值，所以针对AdaBoost提出的采样方法不能直接应用。幸运的是，我们注意到GBDT中每个数据实例的梯度为我们提供了有用的数据采样信息。也就是说，如果一个样本与一个小的梯度相关联，那么这个样本的训练误差就很小，而且它已经经过了良好的训练。一个简单的想法是使用小的梯度丢弃那些数据样本。但是这样做会改变数据分布，会影响学习模型的准确性。为了避免这一问题，我们提出了一种新的基于梯度的单边采样(GOSS)方法。

* GOSS keeps all the instances with large gradients and performs random sampling on the instances with small gradients. In order to compensate the influence to the data distribution, when computing the information gain, GOSS introduces a constant multiplier for the data instances with small gradients (see Alg. 2). Specifically, GOSS firstly sorts the data instances according to the absolute value of their gradients and selects the top a × 100% instances. Then it randomly samples b × 100% instances from the rest of the data. After that, GOSS amplifies the sampled data with small gradients by a constant 1−a when calculating the information gain. By doing so, we put more focus on the under-trained b instances without changing the original data distribution by much.

* GOSS保留所有梯度大的样本，并对梯度小的样本进行随机抽样。为了补偿数据分布的影响,计算信息增益时,高斯数据实例的介绍一个常数乘法器与小梯度(见Alg.2)。具体来说,戈斯首先排序数据样本根据其梯度的绝对值并选择顶部实例 Top a×100%。然后从剩下的数据中随机抽取b×100%的实例。之后，GOSS在计算信息增益时，用小梯度将采样数据放大1−a。通过这样做，我们更多地关注训练不足的b实例，而不改变原始数据分布。

### 3.2 理论分析

* GBDT uses decision trees to learn a function from the input space X s to the gradient space G [1]. Suppose that we have a training set with n i.i.d. instances {x1, · · · , xn}, where each xi is a vector with dimension s in space Xs. In each iteration of gradient boosting, the negative gradients of the loss function with respect to the output of the model are denoted as {g1 , · · · , gn }. The decision tree model splits each node at the most informative feature (with the largest information gain). For GBDT, the information gain is usually measured by the variance after splitting, which is defined as below.

* GBDT使用决策树来学习从输入空间$X^s$到梯度空间G[1]的函数。假设我们有一个包含$n$个i.i.d实例的训练集${(x_1，···，x_n)}$，其中每个$x_i$都是空间$X^s$中的一个维数为s的向量。在梯度助推的每次迭代中，损失函数相对于模型输出的负梯度记为${g_1，···，g_n}$。决策树模型在最有信息的特性(具有最大的信息增益)处拆分每个节点。对于GBDT来说，信息增益通常用分割后的方差来度量，定义如下:

* Definition 3.1 Let O be the training dataset on a fixed node of the decision tree. The variance gain of splitting feature j at point d for this node is defined as

* 定义3.1设O为决策树固定节点上的训练数据集。定义该节点在点d处分割特征j的方差增益为

![avatar](pic/pic_2.png)

* For feature$j$,the decision tree algorithm selects $d_{j}^{*} = argmax x_d V_j (d)$ and calculates the largest gain $V_j (d)$. Then, the data are split according feature $j^*$ at point $d_{j}^{*}$ into the left and right child nodes. 

* 对于特征$j$,决策树算法进行选择 $d_{j}^{*} = argmax x_d V_j (d)$ 以及计算最大的增益 $V_j (d)$. 然后，根据特征对数据进行分割 $j^*$ 在点$d_{j}^{*}$ 到左子节点和右子节点。

* In our proposed GOSS method, first, we rank the training instances according to their absolute values of their gradients in the descending order; second, we keep the top $-a×100\%$ instances with the larger gradients and get an instance subset A; then, for the remaining set $A^c$ consisting $(1 − a) × 100\%$ instances with smaller gradients, we further randomly sample a subset B with size $b × |A^c|$; finally, we split the instances according to the estimated variance gain $\hat{V}_j (d)$ over the subset $A ∪ B$, i.e.,

* 在我们提出的GOSS方法中, 首先，根据训练实例的梯度绝对值，按梯度的降序对训练实例进行排序; 第二，我们保留顶部 $-a×100\%$ 实例，并得到一个实例子集A; 然后，对于剩下的集合 $A^c$ 组成 $(1 − a) × 100\%$ 具有较小梯度的实例, 我们进一步随机抽取a子集B的大小 $b × |A^c|$; 最后，我们根据估计的方差增益分割实例 $\hat{V}_j (d)$ 的子集 $A ∪ B$, i.e.,

![avatar](pic/pic_3.png)

![avatar](pic/pic_4.png)

* Thus, in GOSS, we use the estimated $V ̃j (d)$ over a smaller instance subset, instead of the accurate $Vj (d)$ over all the instances to determine the split point, and the computation cost can be largely reduced. More importantly, the following theorem indicates that GOSS will not lose much training accuracy and will outperform random sampling. Due to space restrictions, we leave the proof of the theorem to the supplementary materials.

* 因此，在GOSS中，我们使用估计值 $V ̃j (d)$ 而不是更小的实例子集 $Vj (d)$ 通过对所有实例进行分界点确定，可大大降低计算成本。更重要的是，下面的定理表明GOSS不会损失太多的训练精度，并且会比随机抽样的效果更好。由于空间的限制，我们把定理的证明留给补充材料。

<br>[返回](#lightGBM的优化)</br>

##### 减少特征上的优化

* In this section, we propose a novel method to effectively reduce the number of features.

* 在本节中，我们提出了一种新的方法来有效地减少特征的数量。

* High-dimensional data are usually very sparse. The sparsity of the feature space provides us a possibility of designing a nearly lossless approach to reduce the number of features. Specifically, in a sparse feature space, many features are mutually exclusive, i.e., they never take nonzero values simultaneously. We can safely bundle exclusive features into a single feature (which we call an exclusive feature bundle). By a carefully designed feature scanning algorithm, we can build the same feature histograms from the feature bundles as those from individual features. In this way, the complexity of histogram building changes from O(#data × #f eature) to O(#data × #bundle), while #bundle << #feature. Then we can significantly speed up the training of GBDT without hurting the accuracy. In the following, we will show how to achieve this in detail.

* 高维数据通常是非常稀疏的。特征空间的稀疏性为我们提供了一种近乎无损的减少特征数量的方法。具体来说，在稀疏特征空间中，许多特征是相互排斥的，即，它们从不同时取非零值。我们可以安全地将排他特性捆绑成一个单独的特性(我们称之为排他特性捆绑)。通过一个精心设计的特征扫描算法，我们可以从特征包中构建出与单个特征相同的特征直方图。这样，直方图构建的复杂性从O(#data×# fature)变为O(#data×#bundle)，而#bundle << #特性。这样就可以在不影响精度的前提下，显著加快GBDT的训练速度。在下面，我们将详细说明如何实现这一点。

* There are two issues to be addressed. The first one is to determine which features should be bundled together. The second is how to construct the bundle.

* 有两个问题需要解决。第一个问题是确定哪些特性应该捆绑在一起。第二个问题是如何构造包。

* Theorem 4.1 The problem of partitioning features into a smallest number of exclusive bundles is NP-hard.

* 定理4.1将特征分割成最小数量的排它包的问题是NP-hard问题。

* Proof: We will reduce the graph coloring problem [25] to our problem. Since graph coloring problem is NP-hard, we can then deduce our conclusion.

* 证明:我们将把图着色问题[25]简化为我们的问题。由于图的着色问题是np -困难的，我们可以推导出我们的结论。

* Given any instance G = (V, E) of the graph coloring problem. We construct an instance of our problem as follows. Take each row of the incidence matrix of G as a feature, and get an instance of our problem with |V | features. It is easy to see that an exclusive bundle of features in our problem corresponds to a set of vertices with the same color, and vice versa

* 给定图着色问题的任意实例G = (V, E)。我们构造问题的一个实例如下。将G的关联矩阵的每一行作为一个特征，得到我们用|V |特征的问题的一个实例。很容易看出，我们的问题中有一个排他的特征集对应于一组颜色相同的顶点，反之亦然

* For the first issue, we prove in Theorem 4.1 that it is NP-Hard to find the optimal bundling strategy, which indicates that it is impossible to find an exact solution within polynomial time. In order to find a good approximation algorithm, we first reduce the optimal bundling problem to the graph coloring problem by taking features as vertices and adding edges for every two features if they are not mutually exclusive, then we use a greedy algorithm which can produce reasonably good results (with a constant approximation ratio) for graph coloring to produce the bundles. Furthermore, we notice that there are usually quite a few features, although not 100% mutually exclusive, also rarely take nonzero values simultaneously. If our algorithm can allow a small fraction of conflicts, we can get an even smaller number of feature bundles and further improve the computational efficiency. By simple calculation, random polluting a small fraction of feature values will affect the training accuracy by at most $O([(1 − γ)n]^{−2/3})$(See Proposition 2.1 in the supplementary materials), where γ is the maximal conflict rate in each bundle. So, if we choose a relatively small γ, we will be able to achieve a good balance between accuracy and efficiency.

* 对于第一个问题，我们在定理4.1中证明了找到最优捆绑策略是NP-Hard，这表明在多项式时间内不可能找到精确解。为了找到一个好的近似算法,我们首先降低图着色问题的最优捆绑问题以特性为顶点和添加边每两个特性如果他们并不是相互排斥的,那么我们使用贪婪算法可以产生良好的结果(一个常数近似比)图着色生产包。此外，我们注意到通常有相当多的特性，尽管不是100%互斥的，但也很少同时接受非零值。如果我们的算法允许一小部分冲突，我们可以得到更小数量的特征包，从而进一步提高计算效率。通过简单的计算,随机污染的一小部分特征值会影响训练精度最多O(((1−γ)n]−2/3)(见命题2.1补充材料),其中γ是最大的冲突率在每个包。所以,如果我们选择一个相对较小的γ,我们将能够实现良好的精度和效率之间的平衡。

* Based on the above discussions, we design an algorithm for exclusive feature bundling as shown in Alg. 3. First, we construct a graph with weighted edges, whose weights correspond to the total conflicts between features. Second, we sort the features by their degrees in the graph in the descending order. Finally, we check each feature in the ordered list, and either assign it to an existing bundle with a small conflict (controlled by γ), or create a new bundle. The time complexity of Alg. 3 is O(#feature2) and it is processed only once before training. This complexity is acceptable when the number of features is not very large, but may still suffer if there are millions of features. To further improve the efficiency, we propose a more efficient ordering strategy without building the graph: ordering by the count of nonzero values, which is similar to ordering by degrees since more nonzero values usually leads to higher probability of conflicts. Since we only alter the ordering strategies in Alg. 3, the details of the new algorithm are omitted to avoid duplication.

* 在上述讨论的基础上，我们设计了Alg. 3所示的排他特征捆绑算法。首先，我们构造一个带有加权边的图，其权值对应于特征之间的总冲突。其次，我们按特征在图中的程度按降序排序。最后,我们检查每个特性在有序列表,并将其分配给一个现有的包与一个小冲突(由γ),或者创建一个新包。Alg. 3的时间复杂度是O(#feature2)，在训练之前只处理一次。当特性的数量不是很大时，这种复杂性是可以接受的，但是如果有数百万个特性，这种复杂性仍然会受到影响。为了进一步提高效率，我们提出了一种不需要构建图的更高效的排序策略:通过非零值的计数排序，这类似于按度排序，因为更多的非零值通常会导致更高的冲突概率。由于我们只改变了Alg. 3中的排序策略，为了避免重复，我们省略了新算法的细节。

* For the second issues, we need a good way of merging the features in the same bundle in order to reduce the corresponding training complexity. The key is to ensure that the values of the original features can be identified from the feature bundles. Since the histogram-based algorithm stores discrete bins instead of continuous values of the features, we can construct a feature bundle by letting exclusive features reside in different bins. This can be done by adding offsets to the original values of the features. For example, suppose we have two features in a feature bundle. Originally, feature A takes value from [0, 10) and feature B takes value [0, 20). We then add an offset of 10 to the values of feature B so that the refined feature takes values from [10, 30). After that, it is safe to merge features A and B, and use a feature bundle with range [0, 30] to replace the original features A and B. The detailed algorithm is shown in Alg. 4.

* 对于第二个问题，我们需要一种好的方法来合并相同包中的特性，以减少相应的训练复杂性。关键是确保可以从功能包中识别原始功能的值。由于基于直方图的算法存储的是离散的bin而不是特征的连续值，我们可以通过让排他的特征驻留在不同的bin中来构造一个特征包。这可以通过向特性的原始值添加偏移量来实现。例如，假设我们在一个特性包中有两个特性。最初，特征A从[0,10]中获取值，而特征B从[0,20)中获取值。然后，我们向特性B的值添加10的偏移量，以便改进的特性从[10,30]中获取值。然后将feature A和B合并，使用一个range[0,30]的feature bundle来代替原来的feature A和B。具体算法见Alg. 4。

* EFB algorithm can bundle many exclusive features to the much fewer dense features, which can effectively avoid unnecessary computation for zero feature values. Actually, we can also optimize the basic histogram-based algorithm towards ignoring the zero feature values by using a table for each feature to record the data with nonzero values. By scanning the data in this table, the cost of histogram building for a feature will change from O(#data) to O(#non_zero_data). However, this method needs additional memory and computation cost to maintain these per-feature tables in the whole tree growth process. We implement this optimization in LightGBM as a basic function. Note, this optimization does not conflict with EFB since we can still use it when the bundles are sparse.

* EFB算法可以将许多排他特征捆绑到更少的密集特征上，有效避免了零特征值的不必要计算。实际上，我们还可以优化基本的基于直方图的算法，通过使用每个特征的表来记录非零值的数据，从而忽略零特征值。通过扫描这个表中的数据，为一个特性构建直方图的成本将从O(#data)变为O(#non_zero_data)。但是，这种方法需要额外的内存和计算成本来在整个树的生长过程中维护这些每个特性的表。我们在LightGBM中实现了这个优化作为一个基本函数。注意，这种优化与EFB并不冲突，因为当bundle是稀疏的时，我们仍然可以使用它。

<br>[返回](#lightGBM的优化)</br>